# GPLVM in Finance

## Imports

In [ ]:
#math and plotting imports
import numpy as np
import pandas as pd
import pystan
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import arviz as az


#CompSci imports
import os
import sys
from contextlib import contextmanager
from datetime import datetime, timedelta
from time import sleep
from hashlib import md5
from zipfile import ZipFile
import pickle

In [ ]:
#helper function imports
from Helpers import (random_weigths, StanModel_cache, vb, calc, DoIt, data_loader_price, data_loader_returns, choose_example_stocks)

## Dictionaries

In [ ]:
#translate model names into stan
model_dict = {
    'linear' = 0,
    'squ_exp' = 1,
    'exp' = 2,
    'matern32' = 3,
    'matern52' = 4
    'squ_exp_lin' = 5,
    'exp_lin' = 6,
    'squ_exp_linadd' = 7,
    'exp_linadd' = 8,
}

#translate data objects for stan
data_dict = {
    'N':N, 
    'D':D, 
    'Q':Q, 
    'Y':Y, 
    'model_number':model_dict[model_name]
}

#ticker list 
tickers = ["MMM", "ABT", "ABBV", "ABMD", "ACN", "ATVI", "ADBE", "AMD", "AAP", "AES", "AMG", "AFL", "A", "APD", "AKAM", "ALK", "ALB", "ARE", "ALXN", "ALGN", "ALLE", "AGN", "ADS", "LNT", "ALL", "GOOGL", "GOOG", "MO", "AMZN", "AMCR", "AEE", "AAL", "AEP", "AXP", "AIG", "AMT", "AWK", "AMP", "ABC", "AME", "AMGN", "APH", "ADI", "ANSS", "ANTM", "AON", "AOS", "APA", "AIV", "AAPL", "AMAT", "APTV", "ADM", "ARNC", "ANET", "AJG", "AIZ", "T", "ADSK", "ADP", "AZO", "AVB", "AVY", "BHGE", "BLL", "BAC", "BK", "BAX", "BBT", "BDX", "BRK-B", "BBY", "BIIB", "BLK", "HRB", "BA", "BWA", "BXP", "BSX", "BMY", "AVGO", "BR", "BF-B", "CHRW", "COG", "CDNS", "CPB", "COF", "CPRI", "CAH", "KMX", "CCL", "CAT", "CBOE", "CBRE", "CBS", "CDW", "CE", "CELG", "CNC", "CNP", "CTL", "CERN", "CF", "SCHW", "CHTR", "CVX", "CMG", "CMI", "CVS", "DHI", "DHR", "DRI", "DVA", "DE", "DAL", "XRAY", "DVN", "FANG", "DLR", "DFS", "DISCA", "DISCK", "DISH", "DG", "DLTR", "D", "DOV", "DOW", "DTE", "DUK", "DRE", "DD", "DXC", "ETFC", "EMN", "ETN", "FLT", "FLIR", "FLS", "FMC", "F", "FTNT", "FTV", "FBHS", "FOXA", "FOX", "BEN", "FCX", "GPS", "GRMN", "IT", "GD", "GE", "GIS", "GM", "GPC", "GILD", "GL", "GPN", "GS", "GWW", "HAL", "HBI", "HOG", "HIG", "HAS", "HCA", "HCP", "HP", "HSIC", "HSY", "HES", "HPE", "HLT", "HFC", "HOLX", "HD", "HON", "HRL", "HST", "HPQ", "HUM", "HBAN", "IDXX", "INFO", "ITW", "ILMN", "IR", "INTC", "ICE", "IBM", "INCY", "IP", "IPG", "IFF", "INTU", "ISRG", "IVZ", "IPGP", "IQV", "IRM", "JKHY", "JEC", "JBHT", "SJM", "JNJ", "JCI", "JPM", "JNPR", "KSU", "K", "KEY", "KEYS", "KMB", "KIM", "KMI", "KLAC", "KSS", "KHC", "KR", "LB", "LHX", "LH", "LRCX", "LW", "LVS", "LEG", "LDOS", "LEN", "LLY", "LNC", "LIN", "LKQ", "LMT", "L", "LOW", "LYB", "MTB", "MAC", "M", "MRO", "MPC", "MKTX", "MAR", "MMC", "MLM", "MAS", "MA", "MKC", "MXIM", "MCD", "MCK", "MDT", "MRK", "MET", "MTD", "MGM", "MCHP", "MU", "MSFT", "MAA", "MHK", "TAP", "MDLZ", "MNST", "MCO", "MS", "MOS", "MSI", "MYL", "NDAQ", "NOV", "NTAP", "NFLX", "NWL", "NEM", "NWSA", "NWS", "NEE", "NLSN", "NKE", "NI", "NBL", "JWN", "NSC", "NTRS", "NOC", "NCLH", "NRG", "NUE", "NVDA", "ORLY", "OXY", "OMC", "OKE", "ORCL", "PCAR", "PKG", "PH", "PAYX", "PYPL", "PNR", "PBCT", "PEP", "PKI", "PRGO", "PFE", "PM", "PSX", "PNW", "PXD", "PNC", "PPG", "PPL", "PFG", "PG", "PGR", "PLD", "PRU", "PEG", "PSA", "PHM", "PVH", "QRVO", "PWR", "QCOM", "DGX", "RL", "RJF", "RTN", "O", "REG", "REGN", "RF", "RSG", "RMD", "RHI", "ROK", "ROL", "ROP", "ROST", "RCL", "CRM", "SBAC", "SLB", "STX", "SEE", "SRE", "SHW", "SPG", "SWKS", "SLG", "SNA", "SO", "LUV", "SPGI", "SWK", "SBUX", "STT", "SYK", "STI", "SIVB", "SYMC", "SYF", "SNPS", "SYY", "TMUS", "TROW", "TTWO", "TPR", "TGT", "TEL", "FTI", "TFX", "TXN", "TXT", "TMO", "TIF", "TWTR", "TJX", "TSCO", "TDG", "TRV", "TRIP", "TSN", "UDR", "ULTA", "USB", "UAA", "UA", "UNP", "UAL", "UNH", "UPS", "URI", "UTX", "UHS", "UNM", "VFC", "VLO", "VAR", "VTR", "VRSN", "VRSK", "VZ", "VRTX", "VIAB", "V", "VNO", "VMC", "WAB", "WMT", "WBA", "DIS", "WM", "WAT", "WEC", "WCG", "WFC", "WELL", "WDC", "WU", "WRK", "WY", "WHR", "WMB", "WLTW", "WYNN", "XEL", "XRX", "XLNX", "XYL", "YUM", "ZBH", "ZION", "ZTS"]


## Specs

In [ ]:
%load_ext autoreload
%autoreload 2

#Parallelization Yes/No?
run_in_parallel = True

#How Many Stocks should be chosen to infer over?
N=10



## Download Data 

In [ ]:
#format: "yyyy-mm-dd"
begin = ""
finish = ""
with suppress_stdout():
    data_loader_price(begin, finish)
    data_loader_returns(begin, finish)

choose_example_stocks()
plot_example_returns()


## Stan Model 

In [ ]:
#specify stan model
gplvm = """

functions{
    matrix cov_linear(vector[] X1, vector[] X2, real sigma){
        int N = size(X1);
        int M = size(X2);
        int Q = num_elements(X1[1]);
        matrix [N,M] K;
        {
            matrix[N,Q] x1;
            matrix[M,Q] x2;
            for (n in 1:N)
        x1[n,] = X1[n]';
            for (m in 1:M)
        x2[m,] = X2[m]';
            K=x1*x2';
        }
        return square(sigma)*K;
    }

    matrix cov_matern32(vector[] X1, vector[] X2, real sigma, real l){
        int N = size(X1);
        int M = size(X2);
        matrix[N,M] K;
        real dist;
        for (n in 1:N)
            for (m in 1:M){
        dist = sqrt(squared_distance(X1[n], X2[m]) + 1e-14);
        K[n,m] = square(sigma)*(1+sqrt(3)*dist/l)*exp(-sqrt(3)*dist/l);
        }
            return K
        }

    matrix cov_matern52(vector[] X1, vector[] X2, real sigma, real l){
        int N = size(X1);
        int M = size(X2);
        matrix[N,M] K;
        real dist;
        for (n in 1:N)
            for (m in 1:M){
        dist = sqrt(squared_distance(X1[n], X2[m]) + 1e-14);
        K[n,m] = square(sigma)*(1+sqrt(5)+(5*square(dist)/(3*square(l)))*dist/l)*exp(-sqrt(5)*dist/l);
        }
            return K
        }

    matrix cov_exp_l2(vector[] X1, vector[] X2, real sigma, real l){
        int N = size(X1);
        int M = size(X2);
        matrix[N,M] K;
        real dist;
        for (n in 1:N)
            for (m in 1:M){
        dist = sqrt(squared_distance(X1[n], X2[m]) + 1e-14);
        K[n,m] = square(sigma)*exp(-0.5/l * dist);
        }
            return K
        }

    matrix cov_exp(vector[] X1, vector[] X2, real sigma, real l){
        int N = size(X1);
        int M = size(X2);
        matrix[N,M] K;
        real dist;
        int Q = rows(X1[1]);
        for (n in 1:N)
            for (m in 1:M){
        dist = 0
        for (i in 1:Q)
            dist = dist + abs(X1[n,i] - X2[m,i]);
        K[n,m] = square(sigma)*exp(-0.5/l * dist);
        }
            return K
        }

    matrix kernel_f(vector[] X1, vector[] X2, real sigma, real l, real a, int model_number, vector diag_stds){
        int N = size(X1)
        int M = size(X2)
        matrix[N,M] K;
        if (model_number==0)
            K = cov_linear(X1, X2, a);
        else if (model_number==1)
            K = cov_exp_quad(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds);
        else if (model_number==2)
            K = cov_exp(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds);
        else if (model_number==3)
            K = cov_matern32(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds);
        else if (model_number==4)
            K = cov_matern52(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds);
    // from here on kernel models are in quad_form_diag (*/+) cov_linear form -> mixture models
        else if (model_number==5)
            K = cov_exp_quad(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds) * cov_linear(X1, X2, a);
        else if (model_number==5)
            K = cov_exp(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds) * cov_linear(X1, X2, a);
        else if (model_number==5)
            K = cov_exp_quad(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds) + cov_linear(X1, X2, a);
        else if (model_number==5)
            K = cov_exp(X1, X2, sigma, l);
            K = quad_form_diag(K, diag_stds) + cov_linear(X1, X2, a);

data {
    int<lower=1> N;
    int<lower=1> D;
    int<lower=1> Q;
    matrix[N,M] Y;
    int<lower=1><upper=8> model_number;
}

transformed data {
    vector[N] mu = rep_vector(0,N);
}

parameters {
    vector [Q] X[N];  //latent space
    real<lower=0> l;   //kernel lengthscale, hyperparameter
    vector<lower=0>[N] diag_stds;   //standard deviation for each stock
    vector<lower=0>[N] noise_std;   //observation noise
    real<lower=0> alpha;    // kernel standard deviation for a linear kernel function
}

transformed parameters{
    real R2=0
    matrix[N,N] L;
    {
        matrix[N,N] K = kernel_f(X, X, 1., kernel_lengthscale, alpha, model_number, diag_stds);
        for (n in 1:N)
            K[n,n] = K[n,n] + pow(noise_std[n], 2) + 1e-14;
        L = cholesky_decompose(K);
        R2 = sum(1- square(noise_std) ./diagonal(K)) /N;
    }
}

model {
    for (n in 1:N)
        X[n] ~ normal(0,1);
    diag_stds ~ normal(0, .5);
    noise_std ~ normal(0, .5);
    kernel_lengthscale ~ inv_gamma(3.0, 1.0);
    alpha ~ inv_gamma(3.0, 1.0)

    for (d in 1:D)
        col(Y,d) ~ multi_normal_cholesky(mu, L);
}

generated quantities {
    real log_likelihood = 0;
    real R2_hat_N = 0;
    vector[N] R2_hat_vec_N;
    
    matrix[N,N] K = kernel_f(X, X, 1., kernel_lengthscale, alpha, model_number, diag_stds);
    
    for (d in 1:D)
        log_likelihood = log_likelihood + multi_normal_cholesky_lpdf(col(Y, d) | mu, L);

    {
        matrix[N,N] K_noise = K;
        matrix[N,D] Y_hat;
        matrix[N,D] residue;

        for (n in 1:N)
            k_noise[n,n] = K_noise[n,n] + pow(noise_std[n], 2) + 1e-14;
        Y_hat = K * mdivide_left_spd(K_noise, Y);
        residue = Y - Y_hat;
        for (n in 1:N)
            R2_hat_vec_N[n] = 1 - sum(square(row(resid,n))) / (sum(square(row(Y,n)-mean(row(Y,n))));
        R2_hat_N = mean(R2_hat_vec_N);
        K = K_noise;   // include non-isotropic noise to output
    }
}

"""

In [ ]:
#compile stan model
stan_model = StanModel_cache(model_code=gplvm)

## Calculations 

In [ ]:
#train the model
calc(model_name= , Q= , num= )


DoIt(run_in_parallel)